# Dice Simulation

JMA July 2024

A simulation based on Kelly-investment strategy to demonstrate sequential hedging and risk taking

In [36]:
import pickle
from pathlib import Path
import numpy as np
from numpy.random import default_rng
import pandas as pd
import seaborn as sns

from bokeh.plotting import figure, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, VBar, Span
from bokeh.io import output_notebook
output_notebook()

rng = default_rng(seed=101)

DICE_SIDES = 60
DICE_THRESHOLD = 45 
OCTAVES = [pow(2, k) for k in range(1,8)]
NUM_PERIODS = OCTAVES[-1]
WIN_BETA_PRIORS = {"a": 45, "b":(60-45)}
SIMULATION_COUNT = 1000
PKL_FILENAME = f'trajectories_{NUM_PERIODS}_by_{SIMULATION_COUNT}.pkl'


Loading BokehJS ...

In [37]:
OCTAVES

[8, 16, 32, 64, 128]

In [38]:
def beta_mean(beta):
    return beta['a']/(beta['a'] + beta['b'])

def fair_win_probability(sides=DICE_SIDES, win_thresh= DICE_THRESHOLD):
    "Probability of a winning investment"
    fair_win = win_thresh / sides
    return fair_win

fair_win_p = fair_win_probability()
f'True probability of an investment win: {fair_win_p:.3},  Prior {beta_mean(WIN_BETA_PRIORS):.3}'

'True probability of an investment win: 0.75,  Prior 0.75'

In [39]:
def win_prob_estimate(win_or_loss, priors):
    'Assume a beta distribution. Do an update assuming a binomial process'
    if win_or_loss == 1:
        priors['a'] +=1
    else:
        priors['b'] +=1
    beta_mean = priors['a'] /(priors['a']+ priors['b'])
    return beta_mean, priors

def investment_process(true_win_p, rounds=NUM_PERIODS):
    'A sequence of 1s and 0s, with win == 1, loss = 0. For the true win stochastic process.'
    return rng.binomial(1, true_win_p, rounds)

def beta_update(wins, prior=WIN_BETA_PRIORS, rounds=NUM_PERIODS):
    'For the unobserved actual win sequence  compute the observed, estimated win prob update.'
    win_estimates = np.zeros(rounds)
    for k, a_win in enumerate(wins):
        win_estimates[k], prior = win_prob_estimate(a_win, prior)
    return win_estimates, prior
    
wins = investment_process(fair_win_p)
win_p_estimates, prior = beta_update(wins)
f'Beta parameters: {prior}, mean: {beta_mean(prior):.4}'

"Beta parameters: {'a': 143, 'b': 45}, mean: 0.7606"

In [40]:
def kelly_rule(p):
    'The fraction to bet for an assumed win probability'
    # This works as a vectorized function 
    return ((2*p - 1) * (p > 0.5))


def log_wealth_trajectory(wins, win_p_estimate):
    """
    Use the estimated win probability update as the bet fraction, using the Kelly rule
    """
    # The log increment for a win is different than for a loss -- is that a problem? 
    log_inc = np.log(1+ (2*wins-1) * kelly_rule(win_p_estimate))
    trajectory = np.cumsum(log_inc)
    return trajectory

In [41]:
estimate_cds = ColumnDataSource(pd.DataFrame({'estimate': win_p_estimates}).reset_index())
pb= figure(
        title="Running Bayes estimate of win probability. ",
        y_range=(min(0.98 *fair_win_p, np.min(win_p_estimates)), max(1.02 *fair_win_p, np.max(win_p_estimates))),
        width = 900, height = 480)
pb.line(x='index', y='estimate', source=estimate_cds)
actual_p = Span(location=fair_win_p, dimension='width', line_dash='dotdash')
pb.renderers.extend([actual_p])


In [42]:
a_trajectory = log_wealth_trajectory(wins, win_p_estimates)
trajectory_cds = ColumnDataSource(pd.DataFrame({'trajectory': a_trajectory}).reset_index())
pt= figure(
        title="Log value over multiple rounds. ",
        #y_range=(min(0.98 *fair_win_p, np.min(win_p_estimate[0])), max(1.02 *fair_win_p, np.max(win_p_estimate[0]))),
        width = 900, height = 480)
pt.line(x='index', y='trajectory', source=trajectory_cds, line_color='darkred')
# actual_p = Span(location=fair_win_p, dimension='width', line_dash='dotdash')
# p.renderers.extend([actual_p])
show(column(pb,pt))

In [43]:



# Careful to reset the prior for each simulation
def run_simulations(fair_win_p, prior=WIN_BETA_PRIORS, simulations=SIMULATION_COUNT):
    """ 
"""
    trajectories = np.zeros((NUM_PERIODS, simulations ))
    for a_sim in range(simulations):
        wins = investment_process(fair_win_p)
        win_p_estimates, prior =beta_update(wins, prior)
        a_trajectory = log_wealth_trajectory(wins, win_p_estimates)
        print('.', end='')
        trajectories[:,a_sim] = a_trajectory

    return trajectories



In [44]:
#SIMULATION_COUNT = 10
simulated_trajectories = run_simulations(fair_win_p, simulations=SIMULATION_COUNT)
pd.DataFrame(simulated_trajectories).describe()

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
count,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,...,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000
mean,5.539820,12.016061,8.983074,7.513646,12.093705,8.632131,6.133123,9.980404,9.717155,14.198879,...,5.031379,4.039497,7.257004,4.887019,6.718190,11.722328,9.780944,7.753022,12.887978,7.615973
std,3.968196,6.585134,3.839862,4.303829,6.437788,5.271312,5.927455,5.173152,6.184307,6.683346,...,4.392141,4.063499,5.713666,2.971094,4.508836,6.621690,7.058855,5.183103,8.103585,3.707003
min,-0.356646,0.402306,0.418119,0.412999,0.114800,-0.884215,-2.577334,0.413116,-0.076980,-0.718064,...,-0.976942,-2.120637,-0.954607,-0.285971,-1.547860,0.118735,-1.381030,-0.572027,-0.334833,0.404718
25%,1.588851,5.624959,6.917183,2.928421,5.819692,3.373022,0.523768,5.251208,3.024473,9.071167,...,0.871914,0.310856,1.254202,2.220924,4.117892,6.464873,2.942621,2.353039,6.150082,4.924093
50%,5.936100,13.107076,9.510032,7.747671,13.049101,9.976696,5.074804,10.863961,12.374107,16.773871,...,3.728725,2.634399,8.097100,5.594090,6.004238,11.883394,10.182249,8.061761,12.956256,6.502002
75%,8.671605,17.224885,11.192828,10.815993,17.896985,12.119217,12.747593,14.556231,14.194066,19.045404,...,9.617275,8.243406,10.424774,7.229887,10.732265,16.782865,15.468859,11.522258,20.976936,10.527575
max,13.676897,22.788850,15.722327,16.838374,22.068633,16.795908,15.662012,17.917361,19.349081,23.108243,...,11.888121,11.602757,20.029739,10.864190,13.626639,24.409214,21.864713,18.530354,25.676195,15.530192


In [45]:
# Define 2 std limits for trajectories.
def std_dev_limits(slice, limit_fraction = 0.05):
    h_bar, h_edges = np.histogram(slice, bins = 100, density=True)
    h_cum_bar = np.cumsum(h_bar *(h_edges[1] - h_edges[0]))
    edges = h_edges[:-1]   # better to use the midpoints
    lower = edges[h_cum_bar < limit_fraction ][-1]
    upper = edges[h_cum_bar > 1-limit_fraction ][0]
    mid = edges[h_cum_bar <=0.5][-1]
    return lower, mid, upper

trajectory_extremes = pd.DataFrame(None, columns=['lower', 'mid', 'upper'])
trajectory_extremes.loc[1] = (0,0,0)
for octave in OCTAVES:
    print(octave, end='\t')
    trajectory_extremes.loc[octave] = std_dev_limits(simulated_trajectories[octave-1,:])



8	16	32	64	128	

In [50]:

# The lower, mid and upper quantiles
te_src = ColumnDataSource(trajectory_extremes.reset_index())

# Plot just some of the trajectories. 
some_trajectories = np.random.choice(range(SIMULATION_COUNT), 20, replace=False)
# Every 16th point leaves 256 points
sub_samples = list(range(0,OCTAVES[-1], OCTAVES[0]))

p= figure(
    title="Selected Trajectories ",
    width = 1000, height = 600, 
    background_fill_color="#fafafa",
    x_axis_type="log",
    x_axis_label = 'Run length',
    y_axis_label = 'Accumulated wealth proportion')

# Downsample the trajectories for efficiency plotting. 
short_trajectories = simulated_trajectories[sub_samples, :]
sim_df = pd.DataFrame(short_trajectories[:, some_trajectories], index = sub_samples)
sim_df.columns = [str(k) for k in sim_df.columns]
sim_src = ColumnDataSource(sim_df)
for k in sim_df.columns:
    p.line(x='index', y=str(k), source=sim_src, line_alpha=0.4, line_color='grey')

p.line(x='index', y='lower', color='red', line_dash='dashed', line_width = 3, source=te_src)
p.line(x='index', y='mid', color='blue', line_width = 3, source=te_src)
p.line(x='index', y='upper', color='red', line_dash='dashed', line_width = 3, source=te_src)


    

show(p)

In [47]:
def bokeh_hist(sample, x_range, title= '', bin_ct=30):
    'Return a graphic histogram object'
    # Find the range of all samples. 
    fig = figure(width= 380, height=360,  title=title, x_range=x_range )
    bars, bin_edges = np.histogram(sample, bins=bin_ct)
    # Split into below and above zero. 
    neg_bin_edges = bin_edges[bin_edges <= 0]
    pos_bin_edges = bin_edges[len(neg_bin_edges):]
    neg_bars = bars[:len(neg_bin_edges)]
    pos_bars = bars[len(neg_bin_edges):]
    neg_bin_edges  = np.append(neg_bin_edges, pos_bin_edges[0])
    LINE_ARGS = dict(line_color=None)
    # Define the sides of each "box" in the histogram plot
    fig.quad(bottom=0, left=neg_bin_edges[:-1], right=neg_bin_edges[1:], top=neg_bars, color='purple', **LINE_ARGS)
    fig.quad(bottom=0, left=pos_bin_edges[:-1], right=pos_bin_edges[1:], top=pos_bars, color='limegreen', **LINE_ARGS)
    return fig

In [48]:
figures = []
xmin, xmax = simulated_trajectories.min(), simulated_trajectories.max()
for octave in OCTAVES:
    figures.append(bokeh_hist(simulated_trajectories[octave-1,:], x_range=(xmin, xmax), title= f'{octave} runs'))
show(column(row(*figures[:2]), 
            row(*figures[2:4]),
            row(*figures[4:6]),
            row(*figures[6:8]),
            row(*figures[8:10]),
            row(*figures[10:])))

In [49]:

if not Path(PKL_FILENAME).exists():
    pickle.dump(simulated_trajectories, open(PKL_FILENAME, 'wb'))
else:
    results = pickle.load(open(PKL_FILENAME, 'rb'))